# Steganography Demo

This notebook demonstrates how to use the steganography techniques implemented in this repository.

In [ ]:
import sys
import os
sys.path.append('..')

from src.steganography import lsb_encode, lsb_decode, dct_encode, dct_decode
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

## 1. LSB Steganography

In [ ]:
# Load the cover and secret images
cover_image_path = '../images/person_mask.jpg'
secret_image_path = '../images/me.png'

cover = Image.open(cover_image_path)
secret = Image.open(secret_image_path).convert('1')  # Convert to binary

# Display original images
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(cover, cmap='gray')
plt.title('Cover Image')
plt.subplot(122)
plt.imshow(secret, cmap='gray')
plt.title('Secret Image')
plt.tight_layout()
plt.show()

In [ ]:
# Encode the secret image into the cover image
encoded_img = lsb_encode(cover, secret)

# Display encoded image
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(cover, cmap='gray')
plt.title('Original Cover Image')
plt.subplot(122)
plt.imshow(encoded_img, cmap='gray')
plt.title('Encoded Image (LSB)')
plt.tight_layout()
plt.show()

In [ ]:
# Decode the secret image
decoded_img = lsb_decode(encoded_img)

# Resize decoded image to original secret image size if needed
decoded_img = decoded_img.resize(secret.size)

# Display decoded image
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(secret, cmap='gray')
plt.title('Original Secret Image')
plt.subplot(122)
plt.imshow(decoded_img, cmap='gray')
plt.title('Decoded Secret Image')
plt.tight_layout()
plt.show()

## 2. DCT Steganography

In [ ]:
# Load the cover and secret images
cover_image = cv2.imread(cover_image_path, cv2.IMREAD_GRAYSCALE)
secret_image = cv2.imread(secret_image_path, cv2.IMREAD_GRAYSCALE)

# Display original images
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(cover_image, cmap='gray')
plt.title('Cover Image')
plt.subplot(122)
plt.imshow(secret_image, cmap='gray')
plt.title('Secret Image')
plt.tight_layout()
plt.show()

In [ ]:
# Encode using DCT
scale_factor = 0.1  # Controls visibility of the hidden image
stego_image = dct_encode(cover_image, secret_image, scale_factor=scale_factor)

# Display encoded image
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(cover_image, cmap='gray')
plt.title('Original Cover Image')
plt.subplot(122)
plt.imshow(stego_image, cmap='gray')
plt.title('Encoded Image (DCT)')
plt.tight_layout()
plt.show()

In [ ]:
# Decode the secret image
decoded_secret = dct_decode(stego_image, cover_image, scale_factor=scale_factor, 
                            secret_shape=secret_image.shape)

# Display decoded image
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(secret_image, cmap='gray')
plt.title('Original Secret Image')
plt.subplot(122)
plt.imshow(decoded_secret, cmap='gray')
plt.title('Decoded Secret Image')
plt.tight_layout()
plt.show()

## Comparison of Image Quality

Let's calculate the PSNR (Peak Signal-to-Noise Ratio) to compare the quality of the encoded images.

In [ ]:
def calculate_psnr(original, encoded):
    """Calculate Peak Signal-to-Noise Ratio between two images."""
    if isinstance(original, Image.Image):
        original = np.array(original)
    if isinstance(encoded, Image.Image):
        encoded = np.array(encoded)
    
    mse = np.mean((original - encoded) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Calculate PSNR for LSB method
psnr_lsb = calculate_psnr(np.array(cover), np.array(encoded_img))
print(f"PSNR for LSB steganography: {psnr_lsb:.2f} dB")

# Calculate PSNR for DCT method
psnr_dct = calculate_psnr(cover_image, stego_image)
print(f"PSNR for DCT steganography: {psnr_dct:.2f} dB")